# 📈 Results Visualization

Advanced visualization of results and trajectories.

**Topics:**
- Trajectory visualization (2D/3D)
- Comparison plots
- Statistical visualizations
- Interactive dashboards

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from hybrid_gcs.visualization import TrajectoryVisualizer

print("✅ Visualization modules loaded!")

## Generate Trajectory Data

In [ ]:
# Generate sample trajectories
def generate_trajectory(start, goal, num_points=50, smoothness=0.3):
    """Generate smooth trajectory from start to goal."""
    t = np.linspace(0, 1, num_points)
    trajectory = np.zeros((num_points, 3))
    
    for i in range(num_points):
        # Bezier curve approximation
        s = t[i]
        trajectory[i] = (1 - s)**2 * start + 2 * (1 - s) * s * (start + goal) / 2 + s**2 * goal
        # Add slight noise for realism
        trajectory[i] += np.random.randn(3) * smoothness
    
    return trajectory

# Generate multiple trajectories
start = np.array([0, 0, 0])
goal = np.array([5, 5, 5])

trajectories = {
    'GCS': generate_trajectory(start, goal, smoothness=0.05),
    'RL': generate_trajectory(start, goal, smoothness=0.15),
    'Hybrid': generate_trajectory(start, goal, smoothness=0.08),
}

print("✅ Generated 3 sample trajectories")

## 2D Trajectory Visualization

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

colors = {'GCS': 'green', 'RL': 'blue', 'Hybrid': 'red'}

for idx, (method, traj) in enumerate(trajectories.items()):
    ax = axes[idx]
    
    # XY plane
    ax.plot(traj[:, 0], traj[:, 1], 'o-', color=colors[method], linewidth=2, markersize=4, label=method)
    ax.plot(start[0], start[1], 'go', markersize=12, label='Start')
    ax.plot(goal[0], goal[1], 'r*', markersize=15, label='Goal')
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title(f'{method} - XY Plane')
    ax.legend()
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')

plt.tight_layout()
plt.show()

## 3D Trajectory Visualization

In [ ]:
fig = plt.figure(figsize=(15, 5))

for idx, (method, traj) in enumerate(trajectories.items()):
    ax = fig.add_subplot(1, 3, idx + 1, projection='3d')
    
    # Plot trajectory
    ax.plot(traj[:, 0], traj[:, 1], traj[:, 2], 'o-', 
            color=colors[method], linewidth=2, markersize=4, label=method)
    
    # Plot start and goal
    ax.scatter(*start, color='green', s=200, marker='o', label='Start', edgecolors='darkgreen', linewidth=2)
    ax.scatter(*goal, color='red', s=300, marker='*', label='Goal', edgecolors='darkred', linewidth=2)
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title(f'{method} - 3D Path')
    ax.legend()

plt.tight_layout()
plt.show()

## Trajectory Metrics Comparison

In [ ]:
# Compute trajectory metrics
def compute_metrics(trajectory):
    """Compute trajectory metrics."""
    # Path length
    diffs = np.diff(trajectory, axis=0)
    path_length = np.sum(np.linalg.norm(diffs, axis=1))
    
    # Smoothness (curvature)
    if len(trajectory) > 2:
        vel = np.diff(trajectory, axis=0)
        acc = np.diff(vel, axis=0)
        smoothness = np.sum(np.linalg.norm(acc, axis=1))
    else:
        smoothness = 0
    
    # Efficiency (straight line distance vs actual)
    straight_dist = np.linalg.norm(trajectory[-1] - trajectory[0])
    efficiency = straight_dist / path_length if path_length > 0 else 0
    
    return {
        'path_length': path_length,
        'smoothness': smoothness,
        'efficiency': efficiency,
    }

metrics = {method: compute_metrics(traj) for method, traj in trajectories.items()}

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

methods = list(metrics.keys())
colors_list = [colors[m] for m in methods]

# Path length
path_lengths = [metrics[m]['path_length'] for m in methods]
axes[0].bar(methods, path_lengths, color=colors_list, alpha=0.7, edgecolor='black', linewidth=2)
axes[0].set_ylabel('Path Length')
axes[0].set_title('Path Length (Lower is Better)')
axes[0].grid(True, alpha=0.3, axis='y')

# Smoothness
smoothness = [metrics[m]['smoothness'] for m in methods]
axes[1].bar(methods, smoothness, color=colors_list, alpha=0.7, edgecolor='black', linewidth=2)
axes[1].set_ylabel('Acceleration Magnitude')
axes[1].set_title('Smoothness (Lower is Better)')
axes[1].grid(True, alpha=0.3, axis='y')

# Efficiency
efficiency = [metrics[m]['efficiency'] for m in methods]
axes[2].bar(methods, efficiency, color=colors_list, alpha=0.7, edgecolor='black', linewidth=2)
axes[2].set_ylabel('Efficiency Ratio')
axes[2].set_title('Path Efficiency (Higher is Better)')
axes[2].grid(True, alpha=0.3, axis='y')
axes[2].set_ylim([0, 1.1])

plt.tight_layout()
plt.show()

# Print detailed metrics
print("Trajectory Metrics Comparison:")
print("=" * 70)
for method, metric_dict in metrics.items():
    print(f"\n{method}:")
    for metric_name, value in metric_dict.items():
        print(f"  {metric_name:12s}: {value:.4f}")

## Method Comparison Radar Chart

In [ ]:
# Normalize metrics for radar chart
def normalize_metrics(metrics_dict):
    """Normalize metrics to 0-1 range."""
    all_values = {}
    for method, metrics in metrics_dict.items():
        for key, value in metrics.items():
            if key not in all_values:
                all_values[key] = []
            all_values[key].append(value)
    
    normalized = {}
    for method, metrics in metrics_dict.items():
        normalized[method] = {}
        for key, value in metrics.items():
            min_val = min(all_values[key])
            max_val = max(all_values[key])
            if key == 'efficiency':  # Higher is better
                normalized[method][key] = (value - min_val) / (max_val - min_val) if max_val > min_val else 0.5
            else:  # Lower is better
                normalized[method][key] = 1 - (value - min_val) / (max_val - min_val) if max_val > min_val else 0.5
    
    return normalized

normalized_metrics = normalize_metrics(metrics)

# Create radar chart
angles = np.linspace(0, 2 * np.pi, len(metrics['GCS']), endpoint=False).tolist()
angles += angles[:1]  # Complete the circle

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(projection='polar'))

metric_names = list(metrics['GCS'].keys())

for method in methods:
    values = [normalized_metrics[method][m] for m in metric_names]
    values += values[:1]  # Complete the circle
    ax.plot(angles, values, 'o-', linewidth=2, label=method, color=colors[method])
    ax.fill(angles, values, alpha=0.15, color=colors[method])

ax.set_xticks(angles[:-1])
ax.set_xticklabels(metric_names)
ax.set_ylim(0, 1)
ax.set_title('Method Comparison Radar', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1))
ax.grid(True)

plt.tight_layout()
plt.show()

## Summary

✅ Advanced visualizations complete:
- 2D trajectory plots
- 3D path visualization
- Metrics comparison
- Radar chart analysis

These tools help understand algorithm performance!
